In [111]:
from datetime import date

import altair as alt
import pandas as pd
import matplotlib.pyplot as plt

In [112]:
df = pd.read_csv('data/konti_capital_punishment_across_sources.csv')
df.rename(
  columns={
  'Year\Source': 'Year'},
  inplace=True)

ldf = pd.melt(df, id_vars=['Year'])
ldf['Year'] = ldf.Year.astype(int)
ldf = ldf[~ldf.value.isna()]
ldf.rename(columns={"variable": "Data source",
                    "value": "Number of executions"},
           inplace=True)
ldf["Year"] = ldf.Year.apply(lambda x: str(x) + "-01-01")
ldf["Year"] = pd.to_datetime(ldf.Year)
ldf.head()

,Year,Data source,Number of executions
10,1991-01-01,Jeannette Chong-Aruldoss,6.0
11,1992-01-01,Jeannette Chong-Aruldoss,21.0
12,1993-01-01,Jeannette Chong-Aruldoss,7.0
13,1994-01-01,Jeannette Chong-Aruldoss,76.0
14,1995-01-01,Jeannette Chong-Aruldoss,73.0


In [113]:
df_grp = ldf.groupby("Year").max().reset_index()
df_grp.head()

,Year,Data source,Number of executions
0,1981-01-01,Tales of Two Cities,3.0
1,1982-01-01,Tales of Two Cities,0.0
2,1983-01-01,Tales of Two Cities,2.0
3,1984-01-01,Tales of Two Cities,2.0
4,1985-01-01,Tales of Two Cities,1.0


In [114]:
sources = alt.Chart(ldf).mark_point(opacity=0.5).encode(
  alt.X('Year',
        scale=alt.Scale(zero=False)
    ),
  y='Number of executions:Q',
  color='Data source:N')
max_line = alt.Chart(df_grp).mark_line().encode(
  alt.X('Year',
        scale=alt.Scale(zero=False)
    ),
  y='Number of executions:Q'
)
sources + max_line

alt.LayerChart(...)

## Visualize gov data

[The Straits Times has published](https://www.straitstimes.com/singapore/more-than-80-of-sporeans-surveyed-believe-death-penalty-has-deterred-offenders-shanmugam?utm_source=Telegram&utm_medium=Social&utm_campaign=STTG) articles stating the governments' evidence that support how capital punishment deters crime.

In [115]:
crimes = {
    'firearm':
      {'df': pd.read_csv("data/scraped_straitstimes_firearm.csv"),
       'date': 1973},
    'kidnapping':
      {'df': pd.read_csv("data/scraped_straitstimes_kidnapping.csv"),
       'date': 1961}}

In [116]:
# Generate hypothetical pasts
crime = 'firearm'
cdf = crimes.get(crime).get('df')
min_year = cdf.year.min()
add_years = 10
years = [min_year - add_years + i + 1 for i in range(add_years)]
scenarios = ['low crime scenario', 'high crime scenario']
good = cdf.cases[-add_years:].to_list()
bad = cdf.cases[:add_years].to_list()
good[-1] = cdf.cases[0]
bad[-1] = cdf.cases[0]

hyp_df = pd.DataFrame(
    {'year': years * len(scenarios),
     'cases': good + bad,
     'hypothetical scenario': [s for s in scenarios for _ in years]})


In [117]:
hyp_df

,year,cases,hypothetical scenario
0,1961,0,low crime scenario
1,1962,0,low crime scenario
2,1963,0,low crime scenario
3,1964,0,low crime scenario
4,1965,0,low crime scenario
5,1966,0,low crime scenario
6,1967,0,low crime scenario
7,1968,0,low crime scenario
8,1969,0,low crime scenario
9,1970,42,low crime scenario


In [118]:
dat = crimes.get(crime)
df = dat.get('df')
line = alt.Chart(df).mark_line().encode(
    x='year:Q',
    y='cases:Q')
vert = alt.Chart(pd.DataFrame([{'year': dat.get('date')}])).mark_rule().encode(x='year')
hyp = alt.Chart(hyp_df).mark_line(opacity=0.5).encode(
    x='year:Q',
    y='cases:Q',
    color='hypothetical scenario:N')
line + vert + hyp

alt.LayerChart(...)

In [119]:
## If we want to normalize by population
pop = pd.read_csv("data/API_SP.POP.TOTL_DS2_en_csv_v2_3918567.csv",
                  skiprows=3)

In [120]:
pop.head(3)

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,Unnamed: 65
0,Aruba,ABW,"Population, total",SP.POP.TOTL,54208.0,55434.0,56234.0,56699.0,57029.0,57357.0,...,102565.0,103165.0,103776.0,104339.0,104865.0,105361.0,105846.0,106310.0,106766.0,NaN
1,Africa Eastern and Southern,AFE,"Population, total",SP.POP.TOTL,130836765.0,134159786.0,137614644.0,141202036.0,144920186.0,148769974.0,...,547482863.0,562601578.0,578075373.0,593871847.0,609978946.0,626392880.0,643090131.0,660046272.0,677243299.0,NaN
2,Afghanistan,AFG,"Population, total",SP.POP.TOTL,8996967.0,9169406.0,9351442.0,9543200.0,9744772.0,9956318.0,...,31161378.0,32269592.0,33370804.0,34413603.0,35383028.0,36296111.0,37171922.0,38041757.0,38928341.0,NaN


In [121]:
is_sg = pop['Country Name'] == 'Singapore'
sg_pop = pop[is_sg]

In [122]:
sg_pop

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,Unnamed: 65
208,Singapore,SGP,"Population, total",SP.POP.TOTL,1646400.0,1702400.0,1750200.0,1795000.0,1841600.0,1886900.0,...,5312437.0,5399162.0,5469724.0,5535002.0,5607283.0,5612253.0,5638676.0,5703569.0,5685807.0,NaN


In [123]:
sg_pop = sg_pop.iloc[:, 4:].T.reset_index()

In [124]:
sg_pop.head(3)

,index,208
0,1960,1646400.0
1,1961,1702400.0
2,1962,1750200.0


## Academic Survey on Public Support for the Death Penalty

https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3122150

In [125]:
interest_df = pd.DataFrame(
    {"Level of interest": 
       ["Very interested or concerned",
        "Interested or concerned",
        "Not very interested or concerned",
        "Not interested or concerned at all"],
     "Percent": [4.5, 44.5, 39.7, 11.3]})
knowledge_df = pd.DataFrame(
    {"Level of knowledge": [
        "Know a great deal",
        "Know something about it",
        "Know little about it",
        "Know nothing about it "],
     "Percent": [2.5, 35.5, 50.7, 11.3]})
support_df = pd.DataFrame(
    {"Level of support": [
       "Strongly in favour",
       "In favour",
       "Oppose",
       "Strongly oppose",
       "Not sure"],
     "Percent": [8.7, 63.2, 22.7, 2.8, 2.6]})


In [126]:
in_favor = (8.7 + 63.2) / 100 * 1500
oppose = (22.7 + 2.8) / 100 * 1500
not_sure = 2.6 / 100 * 1500

In [127]:
import plotly.graph_objects as go

In [128]:
fig = go.Figure(data=[go.Sankey(
    arrangement = "fixed",
    node = dict(
      pad = 15,
      thickness = 50,
      line = dict(color = "black", width = 0.1),
      label = ["Overall", "In favor of dealth penalty", "Oppose", "Not sure",
               "Would still favor it", "Would then oppose it", "Don't know"],
      color = "blue",
      x = [0.1, 0.5, 0.5, 0.5, 0.9, 0.9, 0.9],
      y = [0.5, 0.25, 0.7, 0.85, 0.1, 0.45, 0.65]  
    ),
    link = dict(
      source = [0, 0, 0, 1, 1, 1], # indices correspond to labels, eg A1, A2, A1, B1, ...
      target = [1, 2, 3, 4, 5, 6],
      value = [in_favor, oppose, not_sure,
               in_favor * .501, in_favor * .472, in_favor * 0.027]
  ))])

fig.update_layout(title_text="On drug trafficking - change of support if not proven a better deterrent", font_size=10)
fig.show()

In [129]:
fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 50,
      line = dict(color = "black", width = 0.1),
      label = ["Overall", "In favor of dealth penalty", "Oppose", "Not sure",
               "Would still favor it", "Would then oppose it", "Don't know"],
      color = "blue",
      x = [0.1, 0.5, 0.5, 0.5, 0.9, 0.9, 0.9],
      y = [0.5, 0.25, 0.7, 0.85, 0.1, 0.45, 0.75]
    ),
    link = dict(
      source = [0, 0, 0, 1, 1, 1], # indices correspond to labels, eg A1, A2, A1, B1, ...
      target = [1, 2, 3, 4, 5, 6],
      value = [in_favor, oppose, not_sure,
               in_favor * .324, in_favor * .645, in_favor * 0.031]
  ))])

fig.update_layout(title_text="On drug trafficking - change of support if innocent persons have been executed", font_size=10)
fig.show()